In [ ]:
# utility
import os, os.path
import pathlib
import datetime as dt
from tqdm import tqdm

# data
import numpy as np
import pandas as pd

# image
import cv2 as cv2
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

# ML
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout   #, Rescaling, ZeroPadding2D
from keras.models import Sequential, Model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [ ]:
start = dt.datetime.now()

# Parameters


In [ ]:
# Model select: Set True for Xception, False for Inception Resnet V2
use_xception = True

# Image size for Transfer Learning
img_width, img_height = 299, 299
batch_size = 16

# Path to data
data_dir = "../input/plant-seedlings-classification"

# Réduction eventuelle du dataset à sub_samples images par classe
# 200 pour avoir approximativement la moitié du set, 0 pour tout le set
sub_samples = 0  

# Split train/val
split = 0.2

# Use preprocess ? 
# -- Augmente significativement le score d'accuracy ! Significantly increases the accuracy score!
detourage = True     
# -- mauvais impact, en plus d'être très long bad impact, in addition to being very long
nl_means = False     

# Random seed
SEED = 427

# Data

In [ ]:
train_data_dir = data_dir+"/train"
val_data_dir = train_data_dir     #on split train pour validation
test_data_dir = data_dir+"/test"

labels = sorted(os.listdir(train_data_dir))
for label in labels:
    print(label, len([name for name in os.listdir(train_data_dir+'/'+label) if os.path.isfile(os.path.join(train_data_dir+'/'+label, name))]))

data_dir

In [ ]:
count_plot = []
for i in labels:
    count_plot.append(len(os.listdir(os.path.join(train_data_dir,i))))
count_plot = np.array(count_plot)
plt.bar(labels, count_plot)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
data = []
for category_id, category in enumerate(labels):
    for file in os.listdir(os.path.join(train_data_dir, category)):
        data.append(['{}/{}/{}'.format(train_data_dir, category, file), category_id, category])
data = pd.DataFrame(data, columns=['filepath', 'category_id', 'category'])
display(data.head(2))
print(data.shape)

In [ ]:
#data.hist(column='category_id', bins=12)

In [ ]:
test = []
for file in sorted(os.listdir(test_data_dir)):
    test.append(['{}/{}'.format(test_data_dir, file), file])
test = pd.DataFrame(test, columns=['filepath', 'file'])
display(test.head(2))
print(test.shape)

## Sub sample

In [ ]:
#Sub-sample for faster testing and avoiding memory overflow
if sub_samples > 0:
    data = pd.concat([data[data['category'] == c][:sub_samples] for c in labels])
    data = data.sample(frac=1)
    data.index = np.arange(len(data))
    display(data.head(2))
    print(data.shape)

## Split train/val

In [ ]:
from sklearn.model_selection import train_test_split

stratified_split = True ##because inbalanced data
if stratified_split:
    df_train_x, df_val_x, train_y, val_y = train_test_split(data['filepath'], data['category_id'], test_size=split, random_state=SEED, stratify=data['category_id'])
    print(df_train_x.shape, df_val_x.shape, train_y.shape, val_y.shape)
else:
    np.random.seed(seed=SEED)
    rnd = np.random.random(len(data))
    train_idx = rnd > split
    valid_idx = rnd <= split
    train_y = data.loc[train_idx, 'category_id'].values
    val_y = data.loc[valid_idx, 'category_id'].values
    # df_... contiennent les filenames
    df_train_x = data.loc[train_idx, 'filepath'].values
    df_val_x = data.loc[valid_idx, 'filepath'].values

print("{} images for train".format(len(df_train_x)))
print("{} images for validation".format(len(df_val_x)))
'''
np.random.seed(seed=SEED)
rnd = np.random.random(len(data))
train_idx = rnd > split
valid_idx = rnd <= split
train_y = data.loc[train_idx, 'category_id'].values
val_y = data.loc[valid_idx, 'category_id'].values
# df_... contiennent les filenames
df_train_x = data.loc[train_idx, 'filepath'].values
df_val_x = data.loc[valid_idx, 'filepath'].values

print("{} images for train".format(len(df_train_x)))
print("{} images for validation".format(len(df_val_x)))
'''



# Preprocess spécifique: par la couleur, vu que l'herbe est verte !
# Specific preprocess: by color, since the grass is green!

On converti l'image en HSV (Hue-Saturation-Value) afin de pouvoir créer un masque des zones "vertes", qui est ensuite appliqué à l'image pour la détourer

We convert the image to HSV (Hue-Saturation-Value) in order to be able to create a mask of the "green" areas, which is then applied to the image to clip it
   ![HSV](https://www.mathworks.com/help/images/hsvcone.gif)


In [ ]:
def create_mask_for_plant(image):
    sensitivity=35
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_hsv = np.array([60 - sensitivity, 100, 50])
    upper_hsv = np.array([60 + sensitivity, 255, 255])

    mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    return mask

def segment_plant(image):
    mask = create_mask_for_plant(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output

def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp

In [ ]:
def preprocessing_cnn(img, mask_grass=False, nl_means=False):
    if mask_grass:
        img_segmented = segment_plant(img)
        img = sharpen_image(img_segmented)
    if nl_means:
        # Apply non-local means filter on test img
        img = cv2.fastNlMeansDenoisingColored(
            src=img.astype(np.uint8),
            dst=None,
            h=10,
            hColor=10,
            templateWindowSize=7,
            searchWindowSize=21)
    return img

def show_image_preprocc(image):
    image_mask = create_mask_for_plant(image)
    image_segmented = segment_plant(image)
    image_sharpen = sharpen_image(image_segmented)
    image_nlmeans = preprocessing_cnn(image_sharpen, nl_means=True)
    fig, axs = plt.subplots(1, 5, figsize=(20, 20))
    axs[0].imshow(image)
    axs[1].imshow(image_mask)
    axs[2].imshow(image_segmented)
    axs[3].imshow(image_sharpen)
    axs[4].imshow(image_nlmeans)

## Apercu des filtres Overview of filters

    1ère image: original
    2ème image: masque, basé sur le filtre HSV pour selectionner le vert
    3ème image: image originale avec application du masque
    4ème image: application d'un filtre Gaussien sur la 3eme image    
                --> on utilise celle-ci
    5ème image: application d'un Non-local Means filter surr la 4eme image
                --> il prends pas mal de temps et baisse le score final, donc nous ne l'utiliserons pas
                
    1st picture: original
    2nd image: mask, based on HSV filter to select green
    3rd image: original image with mask applied
    4th image: application of a Gaussian filter on the 3rd image
             --> we use this one
    5th image: application of a Non-local Means filter on the 4th image
             --> it takes a lot of time and lowers the final score, so we won't use it

In [ ]:
# Test image to see the changes, from a train sample
class_indix = 1   # 0-11
img_indix = 1    # < num images dans cette classe ! num images in this class
#image_filename = sorted(os.listdir(os.path.join(train_data_dir, labels[class_indix])))[img_indix]
#image_path = os.path.join(train_data_dir, labels[class_indix],image_filename)

list_files_cat = data['filepath'][data['category_id']==class_indix].reset_index(drop=True)
image_path = list_files_cat[img_indix]

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
show_image_preprocc(image)

# traditional ML 

In [ ]:
pure_classifier = {}

def train_predict_score(model, train_x, train_y, val_x, val_y):
    t0 = dt.datetime.now()
    model.fit(train_x, train_y)
    val_preds = model.predict(val_x)
    val_probs = model.predict_proba(val_x)
    score = accuracy_score(val_y, val_preds)
    t = dt.datetime.now()-t0
    print('Train/Predict/Evaluate time: {} sec'.format(t.seconds))
    return val_preds, val_probs, score, t

In [ ]:
'''
nsamples, nx, ny, nrgb = train_x.shape
x_train2 = train_x.reshape((nsamples,nx*ny*nrgb))
print('Train Images shape: {} size: {:,}'.format(x_train2.shape, x_train2.size))

nsamples, nx, ny, nrgb = val_x.shape
x_test2 = val_x.reshape((nsamples,nx*ny*nrgb))
print('Validation Images shape: {} size: {:,}'.format(x_test2.shape, x_test2.size))

'''

raw_train_x = np.zeros((len(df_train_x), 6075), dtype='float32')
for i, file in tqdm(enumerate(df_train_x)):
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (45, 45))
    img = preprocessing_cnn(img, mask_grass=detourage, nl_means=nl_means)
    img = tf.keras.utils.img_to_array(img)
    img = img.flatten()
    x = preprocess_input(np.expand_dims(img, axis=0))
    raw_train_x[i] = x
print('Train Images shape: {} size: {:,}'.format(raw_train_x.shape, raw_train_x.size))

raw_val_x = np.zeros((len(df_val_x), 6075), dtype='float32')
for i, file in tqdm(enumerate(df_val_x)):
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (45, 45))
    img = preprocessing_cnn(img, mask_grass=detourage, nl_means=nl_means)
    img = tf.keras.utils.img_to_array(img)
    img = img.flatten()
    x = preprocess_input(np.expand_dims(img, axis=0))
    raw_val_x[i] = x
print('Validation Images shape: {} size: {:,}'.format(raw_val_x.shape, raw_val_x.size))




In [ ]:
rfc = RandomForestClassifier(n_estimators=500)
val_pred, val_prob, score, t = train_predict_score(rfc, raw_train_x, train_y, raw_val_x, val_y)
print('Validation RandomForestClassifier Accuracy {}'.format(score))

pure_classifier['pure rfc'] = (rfc, val_pred, val_prob, score, t)

In [ ]:
logreg = LogisticRegression(multi_class='multinomial', solver='sag', random_state=SEED, max_iter=10000)
val_pred, val_prob, score, t = train_predict_score(logreg, raw_train_x, train_y, raw_val_x, val_y)
print('Validation Accuracy [LogisticRegression]: {}'.format(score))

pure_classifier['pure logreg'] = (logreg, val_pred, val_prob, score, t)

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 15)
val_pred, val_prob, score, t = train_predict_score(knn, raw_train_x, train_y, raw_val_x, val_y)
print('Validation Accuracy [KNN]: {}'.format(score))

pure_classifier['raw KNN'] = (knn, val_pred, val_prob, score, t)

In [ ]:
def plot_acc_loss(history, metrics = None):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # au cas ou le callback EarlyStopping arrete avant fin des epochs, on prend la taille réelle
    epochs_range = range(len(loss))

    num_subplot = 2
    if metrics is not None:
        num_subplot += len(metrics)

    plt.figure(figsize=(8+2*num_subplot, 8))
    plt.subplot(1, num_subplot, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    if val_acc is not None:    #sometimes errors prevent validation, we still want training
        plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, num_subplot, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    if val_loss is not None:
        plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    if metrics is not None:
        for i, met in enumerate(metrics):
            m = history.history[met]
            val_m = history.history['val_'+met]
            plt.subplot(1, num_subplot, 3+i)
            plt.plot(epochs_range, m, label='Training '+met)
            if val_m is not None:
                plt.plot(epochs_range, val_m, label='Validation '+met)
            plt.legend()
            plt.title('Training and Validation '+met)
        
    plt.show()

In [ ]:
fully_connected = Sequential([
    Flatten(),
    Dense(100, activation='relu'),
    tf.keras.layers.BatchNormalization(trainable = False,axis=1),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dense(len(labels), activation='softmax')
])

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.00004, beta_1=0.9, beta_2=0.999)
fully_connected.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

t0 = dt.datetime.now()
history = fully_connected.fit(
    x = raw_train_x,
    y = train_y,
    steps_per_epoch= train_x.shape[0],
    epochs=30,
    validation_data=(raw_val_x, val_y),
    validation_steps=raw_val_x.shape[0],
    callbacks=[EarlyStopping(monitor='val_loss', patience=5)]
)

fc_valid_preds_res = fully_connected.predict(raw_val_x)
fc_valid_preds = np.argmax(fc_valid_preds_res, axis=1)
fc_valid_probs = np.max(fc_valid_preds_res, axis=1)
fc_score = accuracy_score(val_y, fc_valid_preds)
t = dt.datetime.now()-t0
print('Train/Predict/Evaluate time: {} sec'.format(t.seconds))
plot_acc_loss(history)
print('Validation Fully-connected Accuracy {}'.format(fc_score))

pure_classifier['raw fc'] = (fully_connected, fc_valid_preds, fc_valid_probs, fc_score, t)

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(300, 200,), random_state=SEED, max_iter=1000)
val_pred, val_prob, score, t = train_predict_score(mlp, raw_train_x, train_y, raw_val_x, val_y)
print('Validation Accuracy [CNN]: {}'.format(score))

pure_classifier['raw CNN'] = (mlp, val_pred, val_prob, score, t)

# Random Forest + TPOT

In [ ]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
param = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(param)

In [ ]:
from tpot import TPOTClassifier

tpot_classifier = TPOTClassifier(generations= 5, population_size= 24, offspring_size= 12,
                                 verbosity= 2, early_stop= 12,
                                 config_dict={'sklearn.ensemble.RandomForestClassifier': param}, 
                                 cv = 4, scoring = 'accuracy')
tpot_classifier.fit(raw_train_x,train_y)

In [ ]:
accuracy = tpot_classifier.score(raw_val_x, val_y)
print(accuracy)


In [ ]:
pure_classifier['rfc + TPOT'] = (tpot_classifier, 0, 0, accuracy, 0)

In [ ]:
best_score = 0.0
best_classifier_key = ''
print("classifier \t score \t\t")
for key in pure_classifier:
    score = pure_classifier[key][3]
    #print("{}\t{}\t{}".format(key, score, pure_classifier[key][4]))
    print("{}\t{}\t".format(key, score))

    if score > best_score:
        best_score = score
        best_classifier_key = key        
print("\nBest classifier: {}  - score: {}".format(best_classifier_key, best_score))

# Xception preprocess

Read image with OpenCV, convert to RGB (OpenCV use BGR), resize, custom preprocess, Xception preprocess

In [ ]:
if use_xception:
    from keras.applications.xception import preprocess_input
else:
    from keras.applications.inception_resnet_v2 import preprocess_input

In [ ]:
train_x = np.zeros((len(df_train_x), img_width, img_height, 3), dtype='float32')
for i, file in tqdm(enumerate(df_train_x)):
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_width, img_height))
    img = preprocessing_cnn(img, mask_grass=detourage, nl_means=nl_means)
    img = tf.keras.utils.img_to_array(img)
    x = preprocess_input(np.expand_dims(img, axis=0))
    train_x[i] = x
print('Train Images shape: {} size: {:,}'.format(train_x.shape, train_x.size))

In [ ]:
val_x = np.zeros((len(df_val_x), img_width, img_height, 3), dtype='float32')
for i, file in tqdm(enumerate(df_val_x)):
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_width, img_height))
    img = preprocessing_cnn(img, mask_grass=detourage, nl_means=nl_means)
    img = tf.keras.utils.img_to_array(img)
    x = preprocess_input(np.expand_dims(img, axis=0))
    val_x[i] = x
print('Validation Images shape: {} size: {:,}'.format(val_x.shape, val_x.size))

## Random images from training set

In [ ]:
y_true_s = [labels[y] for y in train_y]

sample_idxs = np.arange(len(train_x))
np.random.shuffle(sample_idxs)
picks = sample_idxs[:12]

fig = plt.figure(1, figsize=(16,12))
for i, r in enumerate(picks):
    plt.subplot(3,4,i+1)
    plt.subplots_adjust(hspace = 0.1, wspace = 0.1)
    plt.imshow(tf.keras.utils.array_to_img(train_x[r]))
    plt.title('{}'.format(y_true_s[r]))
    plt.xticks([]) , plt.yticks([])
    
plt.show()

# Transfer Learning from Xception - Feature Extraction

In [ ]:
# Import Xception trained model
if use_xception:
    transfer_model = tf.keras.applications.xception.Xception(include_top=False, weights='imagenet', pooling='avg', input_shape=(img_height, img_width, 3))
else:
    transfer_model = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))
### !!! Inception a une sortie de shape (n, 8, 8, ###), au lieu du classique (n, ###), du coup y'a que Fully-connected qui gere ca (ou Flatten?)
INPUTOK = True


# Extract Features from dataset 
train_x_bf = transfer_model.predict(train_x, batch_size=batch_size, verbose=1)
val_x_bf = transfer_model.predict(val_x, batch_size=batch_size, verbose=1)
print('Xception train bottleneck features shape: {} size: {:,}'.format(train_x_bf.shape, train_x_bf.size))
print('Xception valid bottleneck features shape: {} size: {:,}'.format(val_x_bf.shape, val_x_bf.size))

# Feature extraction + Classifiers

We test 3 models: RandomForestClassifier, LogisticRegression, and a NeuralNetwork (Multi-Layer Perceptron)

In [ ]:
classifier = {}

def train_predict_score(model, train_x, train_y, val_x, val_y):
    t0 = dt.datetime.now()
    model.fit(train_x, train_y)
    val_preds = model.predict(val_x)
    val_probs = model.predict_proba(val_x)
    score = accuracy_score(val_y, val_preds)
    t = dt.datetime.now()-t0
    print('Train/Predict/Evaluate time: {} sec'.format(t.seconds))
    return val_preds, val_probs, score, t

## RandomForestClassifier

In [ ]:
if INPUTOK:
    rfc = RandomForestClassifier(n_estimators=500)
    val_pred, val_prob, score, t = train_predict_score(rfc, train_x_bf, train_y, val_x_bf, val_y)
    print('Validation Xception+RandomForestClassifier Accuracy {}'.format(score))

    classifier['rfc'] = (rfc, val_pred, val_prob, score, t)

## LogisticRegression

In [ ]:
if INPUTOK:
    logreg = LogisticRegression(multi_class='multinomial', solver='sag', random_state=SEED, max_iter=10000)
    val_pred, val_prob, score, t = train_predict_score(logreg, train_x_bf, train_y, val_x_bf, val_y)
    print('Validation Accuracy [Xception+LogisticRegression]: {}'.format(score))

    classifier['logreg'] = (logreg, val_pred, val_prob, score, t)

## Fully-Connected

In [ ]:
if INPUTOK:
    fully_connected = Sequential([
        Flatten(),
        Dense(100, activation='relu'),
        tf.keras.layers.BatchNormalization(trainable = False,axis=1),
        Dropout(0.2),
        Dense(50, activation='relu'),
        Dense(len(labels), activation='softmax')
    ])
    # Compiler le modèle
    optimizer = tf.keras.optimizers.Nadam(learning_rate=0.00004, beta_1=0.9, beta_2=0.999)
    fully_connected.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    t0 = dt.datetime.now()
    history = fully_connected.fit(
        x = train_x_bf,
        y = train_y,
        steps_per_epoch= train_x.shape[0],
        epochs=30,
        validation_data=(val_x_bf, val_y),
        validation_steps=val_x_bf.shape[0],
        callbacks=[EarlyStopping(monitor='val_loss', patience=5)]
    )

    fc_valid_preds_res = fully_connected.predict(val_x_bf)
    fc_valid_preds = np.argmax(fc_valid_preds_res, axis=1)
    fc_valid_probs = np.max(fc_valid_preds_res, axis=1)
    fc_score = accuracy_score(val_y, fc_valid_preds)
    t = dt.datetime.now()-t0
    print('Train/Predict/Evaluate time: {} sec'.format(t.seconds))
    plot_acc_loss(history)
    print('Validation Xception+Fully-connected Accuracy {}'.format(fc_score))

    classifier['fc'] = (fully_connected, fc_valid_preds, fc_valid_probs, fc_score, t)

## K Nearest Neighbours

In [ ]:
if INPUTOK:
    knn = KNeighborsClassifier(n_neighbors = 15)
    val_pred, val_prob, score, t = train_predict_score(knn, train_x_bf, train_y, val_x_bf, val_y)
    print('Validation Accuracy [Xception+KNN]: {}'.format(score))

    classifier['KNN'] = (knn, val_pred, val_prob, score, t)

## CNN

In [ ]:
if INPUTOK:
    mlp = MLPClassifier(hidden_layer_sizes=(300, 200,), random_state=SEED, max_iter=1000)
    val_pred, val_prob, score, t = train_predict_score(mlp, train_x_bf, train_y, val_x_bf, val_y)
    print('Validation Accuracy [Xception+CNN]: {}'.format(score))

    classifier['CNN'] = (mlp, val_pred, val_prob, score, t)

## SVM

In [ ]:
def svm_helper(model, train_x, train_y, val_x, val_y):
    t0 = dt.datetime.now()
    model.fit(train_x, train_y)
    val_preds = model.predict(val_x)
    # val_probs = model.predict_proba(val_x)
    score = accuracy_score(val_y, val_preds)
    t = dt.datetime.now()-t0
    print('Train/Predict/Evaluate time: {} sec'.format(t.seconds))
    return val_preds, score, t

In [ ]:
if INPUTOK:
    linear = SVC(kernel='linear', C=1, decision_function_shape='ovo', random_state=SEED)
    val_pred, score, t = svm_helper(linear, train_x_bf, train_y, val_x_bf, val_y)
    print(f'Validation Accuracy [Xception+SVM_linear]: {score}')

    classifier['SVM_linear'] = (linear, val_pred, None, score, t)

In [ ]:
if INPUTOK:
    rbf = SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovo', random_state=SEED)
    val_pred, score, t = svm_helper(rbf, train_x_bf, train_y, val_x_bf, val_y)
    print(f'Validation Accuracy [Xception+SVM_rbf]: {score}')

    classifier['SVM_rbf'] = (rbf, val_pred, None, score, t)

In [ ]:
if INPUTOK:
    poly = SVC(kernel='poly', degree=3, C=1, decision_function_shape='ovo', random_state=SEED)
    val_pred, score, t = svm_helper(poly, train_x_bf, train_y, val_x_bf, val_y)
    print(f'Validation Accuracy [Xception+SVM_poly]: {score}')

    classifier['SVM_poly'] = (poly, val_pred, None, score, t)

In [ ]:
if INPUTOK:
    sig = SVC(kernel='sigmoid', C=1, decision_function_shape='ovo', random_state=SEED)
    val_pred, score, t = svm_helper(sig, train_x_bf, train_y, val_x_bf, val_y)
    print(f'Validation Accuracy [Xception+SVM_sig]: {score}')

    classifier['SVM_sig'] = (sig, val_pred, None, score, t)

## StackingClassifier

In [ ]:
if INPUTOK:
    # Create Base Learners
    base_learners = [
                     ('SVM_poly', poly),
                     ('SVM_linear', linear),
                     ('MLP', mlp)
                    ]

    # Initialize Stacking Classifier with the Meta Learner
    clf = StackingClassifier(estimators=base_learners,
                             final_estimator=LogisticRegression(multi_class='multinomial', max_iter=10000, random_state=SEED))

    # Extract score
    val_pred, val_prob, score, t = train_predict_score(clf, train_x_bf, train_y, val_x_bf, val_y)
    print('Validation Accuracy [Xception+StackClf_1]: {}'.format(score))

    classifier['SC1'] = (clf, val_pred, val_prob, score, t)

In [ ]:
if INPUTOK:
    # Create Base Learners
    base_learners = [
                     ('SVM_poly', poly),
                     ('SVM_linear', linear),
                     ('MLP', mlp),
                     ('logreg', logreg)
                    ]

    # Initialize Stacking Classifier with the Meta Learner
    clf = StackingClassifier(estimators=base_learners,
                             final_estimator=LogisticRegression(multi_class='multinomial', max_iter=10000, random_state=SEED))

    # Extract score
    val_pred, val_prob, score, t = train_predict_score(clf, train_x_bf, train_y, val_x_bf, val_y)
    print('Validation Accuracy [Xception+StackClf_2]: {}'.format(score))

    classifier['SC2'] = (clf, val_pred, val_prob, score, t)

In [ ]:
if INPUTOK:
    # Create Base Learners
    base_learners = [
                     ('LogReg', logreg),
                     ('MLP', mlp)
                    ]

    # Initialize Stacking Classifier with the Meta Learner
    clf = StackingClassifier(estimators=base_learners,
                             final_estimator=LogisticRegression(multi_class='multinomial', max_iter=10000, random_state=SEED))

    # Extract score
    val_pred, val_prob, score, t = train_predict_score(clf, train_x_bf, train_y, val_x_bf, val_y)
    print('Validation Accuracy [Xception+StackClf_3]: {}'.format(score))

    classifier['SC3'] = (clf, val_pred, val_prob, score, t)

In [ ]:
if INPUTOK:
    # Create Learners per layer
    layer_one_estimators = [
                            ('SVM_poly', poly),
                            ('MLP', mlp)
                           ]
    layer_two_estimators = [
                            ('logreg', logreg),
                            ('SVM_linear', linear),
                           ]
    layer_two = StackingClassifier(estimators=layer_two_estimators,
                                   final_estimator=LogisticRegression(multi_class='multinomial', max_iter=10000, random_state=SEED))

    # Create Final model by
    clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)

    # Extract score
    val_pred, val_prob, score, t = train_predict_score(clf, train_x_bf, train_y, val_x_bf, val_y)
    print('Validation Accuracy [Xception+StackClf_4]: {}'.format(score))

    classifier['SC4'] = (clf, val_pred, val_prob, score, t)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

for key in classifier:
    # Use predictions
    y_pred = classifier[key][1]

    # Confusion Matrix
    cf_matrix = confusion_matrix(val_y, y_pred, labels=range(len(labels)))

    fig = plt.figure(figsize=(12,10))
    ax = sns.heatmap(cf_matrix, annot=True, fmt='d')
    ax.set_xlabel("Predicted labels")
    ax.set_ylabel("True labels")
    ax.xaxis.set_ticklabels(labels, rotation='vertical')
    ax.yaxis.set_ticklabels(labels, rotation='horizontal')
    plt.title("Confusion Matrix with model Xception+"+key, fontsize=18)
    plt.show()

In [ ]:
best_score = 0.0
best_classifier_key = ''
print("classifier \t score \t\t time")
for key in classifier:
    score = classifier[key][3]
    print("{}\t{}\t{}".format(key, score, classifier[key][4]))
    if score > best_score:
        best_score = score
        best_classifier_key = key        
print("\nBest classifier: {}  - score: {}".format(best_classifier_key, best_score))

# xception -> xception

In [ ]:
#train_x_bf, train_y, val_x_bf, val_y
xception_train_x = np.zeros((len(train_x_bf), img_width, img_height, 3), dtype='float32')
for i, file in tqdm(enumerate(train_x_bf)):
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_width, img_height))
    img = preprocessing_cnn(img, mask_grass=detourage, nl_means=nl_means)
    img = tf.keras.utils.img_to_array(img)
    x = preprocess_input(np.expand_dims(img, axis=0))
    xception_train_x[i] = x
print('Train Images shape: {} size: {:,}'.format(xception_train_x.shape, xception_train_x.size))

In [ ]:
xception_train_x = []
for i in range(len(train_x_bf)):
    new_x = train_x_bf[i].reshape(1,32,32,2)
    xception_train_x.append(new_x)
x = np.vstack(xception_train_x)
print('Train Images shape: {} size: {:,}'.format(x.shape, x.size))

In [ ]:
xception_train_x = []
for i in range(len(val_x_bf)):
    new_x = val_x_bf[i].reshape(1,32,32,2)
    xception_train_x.append(new_x)
x_val_exception = np.vstack(xception_train_x)
print('Train Images shape: {} size: {:,}'.format(x.shape, x.size))

In [ ]:
xception = tf.keras.applications.xception.Xception(include_top=False, weights='imagenet', pooling='avg', input_shape=(32, 32, 2))

train_x_bf = transfer_model.predict(x, batch_size=batch_size, verbose=1)
val_x_bf = transfer_model.predict(val_x, batch_size=batch_size, verbose=1)

val_pred, val_prob, score, t = train_predict_score(xception, x, train_y, val_x_bf, val_y)
print('Validation Xception+RandomForestClassifier Accuracy {}'.format(score))

classifier['rfc'] = (rfc, val_pred, val_prob, score, t)

In [ ]:
print(len(train_x_bf))

In [ ]:
newarr = train_x_bf[0].reshape(32,32,2)
print('Train Images shape: {} size: {:,}'.format(newarr.shape, newarr.size))

In [ ]:
val_x = np.zeros((len(df_val_x), img_width, img_height, 3), dtype='float32')
for i, file in tqdm(enumerate(df_val_x)):
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_width, img_height))
    img = preprocessing_cnn(img, mask_grass=detourage, nl_means=nl_means)
    img = tf.keras.utils.img_to_array(img)
    x = preprocess_input(np.expand_dims(img, axis=0))
    val_x[i] = x
print('Validation Images shape: {} size: {:,}'.format(val_x.shape, val_x.size))

# Random Validation Samples predictions

In [ ]:
# Exemples au hasard dans set validation, avec dernier modele de la confusion matrix
# Random examples in set validation, with latest confusion matrix model

# Inverse transform of encoding
y_true_s = [labels[x] for x in val_y]
y_pred_s = [labels[x] for x in classifier[best_classifier_key][1]]

fig = plt.figure(1, figsize=(20,16))
test_idxs = np.arange(len(val_x))
np.random.shuffle(test_idxs)
picks = test_idxs[:12]
for i, r in enumerate(picks):
    batch_r = r // batch_size
    idx_r = r % batch_size
    plt.subplot(3,4,i+1)
    plt.subplots_adjust(hspace = 0.1, wspace = 0.1)
    plt.imshow(tf.keras.utils.array_to_img(val_x[r]))
    plt.title('Actual = {}\nPredicted = {}'.format(y_true_s[r], y_pred_s[r]), color= 'green' if y_true_s[r] == y_pred_s[r] else 'red')
    plt.xticks([]) , plt.yticks([])
    
plt.show()

## Predict an image class with models

In [ ]:
# example of using a pre-trained model as a classifier
#from keras.preprocessing.image import load_img

image_path = os.path.join(data_dir,'test/0021e90e4.png')
#image_path = os.path.join(data_dir,'train/Small-flowered Cranesbill/003402ea0.png')
#image = load_img(image_path, target_size=(img_height, img_width))
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (img_width, img_height))

# show image
plt.figure(figsize=(8, 4))
plt.subplot(121)
plt.imshow(image)
plt.axis("off")

if detourage or nl_means:
    image = preprocessing_cnn(image, mask_grass=detourage, nl_means=nl_means)
    plt.subplot(122)
    plt.imshow(image)
    plt.axis("off")

img_array = tf.keras.utils.img_to_array(image)
img_array = preprocess_input(tf.expand_dims(img_array, 0))
img_bf = transfer_model.predict(img_array)

for key in classifier:
    prediction = classifier[key][0].predict(img_bf)
    if key == 'fc':
        prediction_fc = np.argmax(prediction, axis=1)
        prediction_proba_fc = np.max(prediction, axis=1)
        print("prediction({}): {}  ({})".format(key, labels[prediction_fc[0]], 100*prediction_proba_fc[0]))
    else:
        prediction_proba = classifier[key][0].predict_proba(img_bf)
        print("prediction({}): {}  ({})".format(key, labels[prediction[0]], 100*np.max(prediction_proba[0])))


# Submission

In [ ]:
test_x = np.zeros((len(test), img_width, img_height, 3), dtype='float32')
for i, file in tqdm(enumerate(test['filepath'])):
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_width, img_height))
    img = preprocessing_cnn(img, mask_grass=detourage, nl_means=nl_means)
    img = tf.keras.utils.img_to_array(img)
    x = preprocess_input(np.expand_dims(img, axis=0))
    test_x[i] = x
print('Test Images shape: {} size: {:,}'.format(test_x.shape, test_x.size))

In [ ]:
test_x_bf = transfer_model.predict(test_x, batch_size=batch_size, verbose=1)
print('Xception test bottleneck features shape: {} size: {:,}'.format(test_x_bf.shape, test_x_bf.size))
test_preds = classifier[best_classifier_key][0].predict(test_x_bf)

In [ ]:
# with fully-connected model, need to get best prediction from probas
if best_classifier_key == 'fc':
    test_preds = np.argmax(test_preds, axis=1)

In [ ]:
test['category_id'] = test_preds
test['species'] = [labels[c] for c in test_preds]
test[['file', 'species']].to_csv('submission.csv', index=False)

In [ ]:
print("Total Notebook time: {} sec".format((dt.datetime.now()-start).seconds))